In [2]:
# import pandas
import pandas as pd
# import numpy
import numpy as np

In [7]:
# read the file from CSV from the suggestions, read cedula as string
df_suggestions = pd.read_csv('../storage/csvs/supercias.consultas_personal_suggestion.csv', low_memory=False, dtype={'cedula': str})

# print head 
print(df_suggestions.head())

                        _id      cedula  suggestion[0][0] suggestion[0][1]  \
0  661984fb662d7276e89f03f6  0100163500               NaN              NaN   
1  661984fb662d7276e89f03f7  0100163534               NaN              NaN   
2  661984fc662d7276e89f03f8  0100163567               NaN              NaN   
3  661984fd662d7276e89f03f9  0100163575               NaN              NaN   
4  661984fd662d7276e89f03fa  0100163542               NaN              NaN   

  suggestion[1][0] suggestion[1][1] suggestion[1][2]  
0              NaN              NaN              NaN  
1              NaN              NaN              NaN  
2              NaN              NaN              NaN  
3              NaN              NaN              NaN  
4              NaN              NaN              NaN  


In [8]:
df_suggestions = df_suggestions.drop('_id', axis=1)
# print head
print(df_suggestions.head())


       cedula  suggestion[0][0] suggestion[0][1] suggestion[1][0]  \
0  0100163500               NaN              NaN              NaN   
1  0100163534               NaN              NaN              NaN   
2  0100163567               NaN              NaN              NaN   
3  0100163575               NaN              NaN              NaN   
4  0100163542               NaN              NaN              NaN   

  suggestion[1][1] suggestion[1][2]  
0              NaN              NaN  
1              NaN              NaN  
2              NaN              NaN  
3              NaN              NaN  
4              NaN              NaN  


In [9]:
# get the rows from the suggestion[0][0] column which is not NaN    
# print columns
# has frist suggestions 
print('has frist suggestions')
print(df_suggestions.loc[~df_suggestions['suggestion[0][1]'].isnull()].shape)
# has second suggestions
print('has second suggestions')
print(df_suggestions.loc[~df_suggestions['suggestion[1][0]'].isnull()].shape)
# has any suggestions
print('has any suggestions')
any_df = df_suggestions.loc[~df_suggestions['suggestion[0][1]'].isnull() | ~df_suggestions['suggestion[1][1]'].isnull()]
print(any_df.shape)
# has both of the the one that have any suggestions
print('has both of the the one that have any suggestions')
both_df = any_df.loc[~any_df['suggestion[0][1]'].isnull() & ~any_df['suggestion[1][1]'].isnull()]
print(both_df.shape)
# print columns the frist suggestions is the same as the second suggestions
print('the frist suggestions is the same as the second suggestions')
print(df_suggestions.loc[df_suggestions['suggestion[0][1]'] == df_suggestions['suggestion[1][1]']].shape)
# print columns the frist suggestions is not the same as the second suggestions
print('get rows that are NaN')
print(df_suggestions.loc[df_suggestions['suggestion[0][1]'].isnull() & df_suggestions['suggestion[1][1]'].isnull()].shape)
# print columns the frist suggestions is not the same as the second suggestions
print('the frist suggestions is not the same as the second suggestions')
print(both_df.loc[both_df['suggestion[0][1]'] != both_df['suggestion[1][1]']].shape)
# this are ususlay the same person with difrent spelling of their names for some reason
#print(both_df.loc[both_df['suggestion[0][1]'] != both_df['suggestion[1][1]']].head())


has frist suggestions
(94438, 6)
has second suggestions
(236, 6)
has any suggestions
(94438, 6)
has both of the the one that have any suggestions
(235, 6)
the frist suggestions is the same as the second suggestions
(184, 6)
get rows that are NaN
(2022767, 6)
the frist suggestions is not the same as the second suggestions
(51, 6)


In [10]:
# get the first suggestion colum
first_suggestion_df = df_suggestions.loc[~df_suggestions['suggestion[0][1]'].isnull()]
# drop the other columns 
first_suggestion_df = first_suggestion_df[['cedula', 'suggestion[0][1]']]
# change the column name suggestion[0][1] to suggestion
first_suggestion_df.columns = ['cedula', 'nombre']
# get the seccond suggestion colum
second_suggestion_df = df_suggestions.loc[~df_suggestions['suggestion[1][1]'].isnull()]
# drop the other columns
second_suggestion_df = second_suggestion_df[['cedula', 'suggestion[1][1]']]
# change the column name suggestion[1][1] to suggestion
second_suggestion_df.columns = ['cedula', 'nombre']
# merge the two dataframes with the cedula column ans the suggestion column
# the merged df should have only one suggestion column with the cedula column
# if there are duplicates of the same cedula the suggestion which is longer should be the one that is kept
merged_df = pd.concat([first_suggestion_df, second_suggestion_df]).drop_duplicates(subset='cedula', keep='last')
# remove duplicates of the same cedula into the same suggestion column
merged_df = merged_df.drop_duplicates(subset='cedula')
# print head
print(merged_df.head())
# print shape
print(merged_df.shape)

        cedula                               nombre
9   0100164433  PERALTA CARDENAS RIGOBERTO NECTARIO
12  0100164359         LAZO PERALTA ERNESTO MARCELO
32  0100225887    UZHCA GUAMAN MANUEL JESUS GONZALO
42  0100125970               GARCIA SINCHE LEOPOLDO
43  0100221720         SARI SEGOVIA OSWALDO IGNACIO
(49612, 2)


In [11]:
# Compare the number of rows
num_rows_all_suggestions = df_suggestions.shape[0]
num_rows_merged_df = merged_df.shape[0]

# Print the comparison
print("All queried cedulas:", num_rows_all_suggestions)
print("Number of cedulas which we got:", num_rows_merged_df)

# Calculate the percentage of the rows that are in the merged_df
percentage = (num_rows_merged_df / num_rows_all_suggestions * 100)
print("Percentage:", round(percentage, 2), "%")


All queried cedulas: 2117205
Number of cedulas which we got: 49612
Percentage: 2.34 %


In [13]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)

# Access the supercias database
db = client['supercias']

# Access the suggestions_01 collection
collection = db['suggestions_01']

# Convert the dataframe to a list of dictionaries
data = merged_df.to_dict(orient='records')

# Insert the data into the collection
collection.insert_many(data)

# Close the MongoDB connection
client.close()
